**install and import libraries**

In [ ]:
pip install python-dateutil lxml requests selectorlib

In [ ]:
pip install requests-html

In [ ]:
import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import pandas as pd

In [4]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser

**Set the path variable**

In [5]:
path_to_files = '/content/'

**Create an Extractor by reading from the YAML file**


In [41]:
e = Extractor.from_yaml_file(path_to_files + 'select_data.yml')

**Create a session object**

In [62]:
session = HTMLSession()

**START SCRAPPING**

_Function to get the product ASINs of the first page of the search results on Amazon's website_

In [9]:
def get_asin_id(search):

  # to handle space-separated terms
  search = search.replace(' ','+')

  # get requested data from the specified URL
  requested_data = session.get(f'https://www.amazon.in/s?k={search}')

  # extract product ASINs from the requested data
  asins = requested_data.html.find('div.s-main-slot div[data-asin]')
  
  product_list = [asin.attrs['data-asin'] for asin in asins if asin.attrs['data-asin']!='']

  return product_list

In [10]:
# Mention the item to be searched. For scraping multiple objects a list can be used
search_item = "tshirt"

In [14]:
product_asin = get_asin_id(search_item)
print(product_asin)

['B072C6S397', 'B01D8GCHAI', 'B08N71YW78', 'B089B36CYP', 'B08GWRCPDX', 'B08CDRGNHH', 'B08HNB3M38', 'B08PV6S2WF', 'B08N714Q2T', 'B08WK6NVK1', 'B07L3KJS2J', 'B0822CSJF6', 'B01GQEGYGC', 'B073X41D5S', 'B08H64VWFY', 'B08N63BSVM', 'B0855HG9HB', 'B07B2Y26C1', 'B08JQJYRZW', 'B07P56DH2C', 'B08TH9VSHW', 'B07M85FYKR', 'B07CK7BPQF', 'B07YDHKXW2', 'B08L9GV85M', 'B079W71K23', 'B08ZN7H1CD', 'B07CQMYL95', 'B08YXNN732', 'B07N89154N', 'B08C2SN8MV', 'B08V5J3SPW', 'B07F2B1SJS', 'B08KGV4M3J', 'B07R6ML9ZM', 'B075F7YC9M', 'B06Y2284FW', 'B0839Q11RD', 'B07HYY77ZT', 'B08X7Z5YB3', 'B07CXXHLY2', 'B082YB8Y41', 'B07NK11GTH', 'B07NJWN4BM', 'B012STPJMY', 'B079RL5JPR', 'B073X4KMFV', 'B01HGW6OBI', 'B089GXVCXN', 'B073X1MR71']


>The URL of the review page of all the items on Amazon is same, except the product ASIN in the URL. So, when we have list of product ASINs, we can easily generate review page urls corresponding to each item.

In [63]:
def extract_review_url(product_asin):
  
  # save the extracted URLs in a text file
  with open(path_to_files + f'urls_{search_item}.txt','w') as url_file:
    for id in product_asin:
      url = f'https://www.amazon.in/product-reviews/{id}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
      print(url)
      url_file.write(url+'\n')

In [18]:
extract_review_url(product_asin)

https://www.amazon.in/product-reviews/B072C6S397/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B01D8GCHAI/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08N71YW78/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B089B36CYP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08GWRCPDX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08CDRGNHH/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08HNB3M38/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08PV6S2WF/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/product-reviews/B08N714Q2T/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://ww

In [42]:
def scrape(url):    
  headers = {
  'authority': 'www.amazon.com',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'dnt': '1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'sec-fetch-site': 'none',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-dest': 'document',
  'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
  }

  # Download the page using requests
  print("Downloading %s"%url)
  r = requests.get(url, headers=headers)
  # Simple check to check if page was blocked (Usually 503)
  if r.status_code > 500:
    if "To discuss automated access to Amazon data please contact" in r.text:
      print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
    else:
      print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
      return None

  # Pass the HTML of the page and create 
  return e.extract(r.text)

In [ ]:
# specify the text file containing URLs as input file and the csv file to be generated as output file

with open(path_to_files + f'urls_{search_item}.txt','r') as urllist, open(path_to_files + f'data_{search_item}.csv','w') as outfile:
  # specify the columns that you want in your csv file
  writer = csv.DictWriter(outfile, fieldnames=["product_name","product_variant","product_image","avg_reviews",
                                              "review_title","star_rating","review_date","review_content",
                                             "helpful","variant","verified","review_images","author_profile","author_name"],quoting=csv.QUOTE_ALL)
  writer.writeheader()

  for url in urllist.readlines():
    scrapped_data = scrape(url)                   # for each url in the urllist, scrap the data

    # Pre-process the required fields in the scrapped data
    if scrapped_data:
      for r in scrapped_data["Reviews"]:
        r["product_name"] = scrapped_data['Product_info']['product_name']
        r["product_variant"] = scrapped_data['Product_info']['product_variant']
        r["product_image"] = scrapped_data['Product_info']['product_image']
        r["avg_reviews"] = scrapped_data['Product_info']['avg_reviews']

        r['star_rating'] = r['star_rating'].split(' out of')[0]

        date = r['review_date'].split('on ')[-1]
        r['review_date'] = dateparser.parse(date).strftime('%d %b %Y')
        
        r['author_profile'] = 'https://www.amazon.in' + r['author_profile']
        if r['helpful']:
          try:
            r['helpful'] = r['helpful'].split('people found this helpful')[0].strip()
          except:
            r['helpful'] = r['helpful'].split('person found this helpful')[0].strip()

        if 'verified' in r:
          if 'Verified Purchase' in r['verified']:
            r['verified'] = 'Yes'
          else:
            r['verified'] = 'Yes'
        
        if r['review_images']:
          r['review_images'] = "\n".join(r['review_images'])
      
        writer.writerow(r)

_Check how the generated csv file looks like_

In [60]:
# load the generated csv file and store it in a dataFrame.

df = pd.read_csv(path_to_files + f'data_{search_item}.csv')
df

,product_name,product_variant,product_image,avg_reviews,review_title,star_rating,review_date,review_content,helpful,variant,verified,review_images,author_profile,author_name
0,Scott International Men's Regular Fit T Shirt ...,Size: 2XL,https://m.media-amazon.com/images/I/41dhbLM0GO...,3.3 out of 5,Never buy this product!,1.0,11 Aug 2017,Very poor quality. I had bought XL size but af...,8,Size: XL,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,Amazon Customer
1,Scott International Men's Regular Fit T Shirt ...,Size: 2XL,https://m.media-amazon.com/images/I/41dhbLM0GO...,3.3 out of 5,1 sleeve was defected,3.0,26 Sep 2017,1 sleeve was defected,6,Size: M,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,Vivek Kumar
2,Scott International Men's Regular Fit T Shirt ...,Size: 2XL,https://m.media-amazon.com/images/I/41dhbLM0GO...,3.3 out of 5,Value for money,5.0,17 Apr 2021,Good quality,NaN,NaN,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,B C Rout
3,Scott International Men's Regular Fit T Shirt ...,Size: 2XL,https://m.media-amazon.com/images/I/41dhbLM0GO...,3.3 out of 5,Too short,3.0,14 May 2021,"Size chart doesn't seems to be accurate, as M ...",NaN,Size: M,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,mahesh kumar ramagiri
4,Scott International Men's Regular Fit T Shirt ...,Size: 2XL,https://m.media-amazon.com/images/I/41dhbLM0GO...,3.3 out of 5,not worth of money,2.0,24 Jul 2017,i would recommend not to purchase this product...,5,NaN,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,jaynt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,AWG ALL WEATHER GEAR Men's Regular Fit Cotton ...,Colour: Navy Blue Melange with Tipping Size: 2XL,https://m.media-amazon.com/images/I/41FyqnWWSz...,3.6 out of 5,I got the wrong colour!,3.0,17 Jan 2021,I've ordered for the navy blue but I got the r...,4,Colour: Navy Blue Melange with Tipping Size: L,Yes,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.in/gp/profile/amzn1.account...,Surendran C K
57,AWG ALL WEATHER GEAR Men's Regular Fit Cotton ...,Colour: Navy Blue Melange with Tipping Size: 2XL,https://m.media-amazon.com/images/I/41FyqnWWSz...,3.6 out of 5,Seriously deserve 5 stars,5.0,13 Apr 2021,I have given it five start because product its...,One person found this helpful,Colour: Solid Grey Size: S,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,Himanshu kandhari
58,AWG ALL WEATHER GEAR Men's Regular Fit Cotton ...,Colour: Navy Blue Melange with Tipping Size: 2XL,https://m.media-amazon.com/images/I/41FyqnWWSz...,3.6 out of 5,A good quality product,5.0,22 Feb 2021,The product fits as expected. It is comfortabl...,One person found this helpful,Colour: Navy Blue Melange with Tipping Size: L,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,huzefa
59,AWG ALL WEATHER GEAR Men's Regular Fit Cotton ...,Colour: Navy Blue Melange with Tipping Size: 2XL,https://m.media-amazon.com/images/I/41FyqnWWSz...,3.6 out of 5,Product is different compare to the picture here,2.0,29 Oct 2020,Product was different compared to what is in t...,3,NaN,Yes,NaN,https://www.amazon.in/gp/profile/amzn1.account...,Vinaya raj
